<p align="center"> 
    
  ## List of postal codes of Canada 
    
</p>

Import library

In [1]:
import requests
import wikipedia as wp
import pandas as pd
from pandas.io.json import json_normalize

tidy up dataframe: set the header and clean the not assigned Borough

In [2]:
html = wp.page("List_of_postal_codes_of_Canada:_M").html().encode("UTF-8")
df = pd.read_html(html, header = 0, index_col = 0)[0]
df_clean = df.loc[(df['Borough'] != 'Not assigned')]
df_clean.reset_index(inplace = True)


creat a new,empty dataframe to emerge all neighbourhoods that belong to the same postcode

In [3]:
Postcode = df_clean.Postcode.unique()
index = [i for i in range(len(Postcode))]
columns = df_clean.columns
df_merge = pd.DataFrame(columns = columns, index = index)

Emerge all neighbourhood that belong to the same post code and change the not assigned neighbourhood to its borough

In [4]:
for i,post in enumerate(Postcode):
    df = df_clean.loc[(df_clean.Postcode == post)]
    df_merge['Postcode'][i] = post
    df_merge['Borough'][i] = df.Borough.values[0]
    if df.Neighbourhood.values[0] == 'Not assigned':
        df_merge['Neighbourhood'][i] = df.Borough.values[0]
    else:
        df_merge['Neighbourhood'][i] = ', '.join(df.Neighbourhood.values)
   

df_merge.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


import geographical data

In [5]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')

create latitude and longitude columns

In [6]:
df_merge['Latitude'] = ''
df_merge['Longitude'] = ''

loop to retreive coordinates

In [7]:
# loop until you get the coordinates
for i, post in enumerate(Postcode):
    latitude = df_geo['Latitude'].loc[df_geo['Postal Code'] == post].values[0]
    longitude = df_geo['Longitude'].loc[df_geo['Postal Code'] == post].values[0]
    df_merge['Latitude'][i] = latitude
    df_merge['Longitude'][i] = longitude

In [8]:
df_merge.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6543,-79.3606
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7185,-79.4648
4,M7A,Queen's Park,Queen's Park,43.6623,-79.3895


import folium to plot the map for later

In [50]:
import folium

get foursquare venues information

In [49]:
CLIENT_ID = '2FN4SC0Z1VIANTEZQEK2AMHGZFSDOURUGJR5ZLLDRIPMPZIT' # your Foursquare ID
CLIENT_SECRET = 'UAZB2SXHUOES3H0CSWOMALP1X2DB0QL5UVNDHYUA21ASWFZ2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 100
radius = 500

latitude = df_merge['Latitude'][0]
longtitude = df_merge['Longitude'][0]
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(CLIENT_ID,
                                                                                                                 CLIENT_SECRET,
                                                                                                                 VERSION,
                                                                                                                 latitude,
                                                                                                                 longitude,
                                                                                                                 radius,
                                                                                                                 LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=2FN4SC0Z1VIANTEZQEK2AMHGZFSDOURUGJR5ZLLDRIPMPZIT&client_secret=UAZB2SXHUOES3H0CSWOMALP1X2DB0QL5UVNDHYUA21ASWFZ2&v=20180605&ll=43.7532586,-79.52099940000001&radius=500'

put json into dataframe

In [25]:
results = requests.get(url).json()
df = pd.DataFrame(results)

,name,categories,lat,lng
0,Pho Com Viet Nam,Vietnamese Restaurant,43.756631,-79.518336
1,Pizza Hut,Pizza Place,43.756340,-79.517818
2,KFC,Fast Food Restaurant,43.756549,-79.519042
3,The Beer Store,Beer Store,43.756094,-79.516239
4,Tim Hortons,Coffee Shop,43.756128,-79.516266


tidy up data

In [51]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [52]:
toronto_venues = getNearbyVenues(names=df_merge['Borough'],
                                   latitudes=df_merge['Latitude'],
                                   longitudes=df_merge['Longitude'])

In [53]:
toronto_venues.head()

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,North York,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,North York,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [54]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
grouped = toronto_onehot.groupby('Borough').mean().reset_index()
grouped.head()

,Borough,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.009009,0.000000,0.000000,0.009009,0.000000,0.000000,0.000000,0.000000,0.009009
1,Downtown Toronto,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.001553,0.001553,0.001553,...,0.002329,0.013199,0.002329,0.000776,0.003882,0.000000,0.006211,0.000776,0.000776,0.002329
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024590
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000,0.000000,0.000000,0.012987
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.013889,0.000000,0.000000,0.000000,0.013889,0.000000,0.000000


In [42]:
from sklearn.cluster import KMeans

In [44]:
kclusters = 5

clustering = grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
clustering.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018018,...,0.000000,0.009009,0.000000,0.000000,0.009009,0.000000,0.000000,0.000000,0.000000,0.009009
1,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.001553,0.001553,0.001553,0.013975,...,0.002329,0.013199,0.002329,0.000776,0.003882,0.000000,0.006211,0.000776,0.000776,0.002329
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024590,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024590
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000,0.000000,0.000000,0.012987
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013889,...,0.000000,0.000000,0.000000,0.013889,0.000000,0.000000,0.000000,0.013889,0.000000,0.000000


In [47]:
grouped.insert(0, 'Cluster Labels', kmeans.labels_)

,Cluster Labels,Borough,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,1,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.009009,0.000000,0.000000,0.009009,0.000000,0.000000,0.000000,0.000000,0.009009
1,0,Downtown Toronto,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.001553,0.001553,...,0.002329,0.013199,0.002329,0.000776,0.003882,0.000000,0.006211,0.000776,0.000776,0.002329
2,0,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024590
3,1,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000,0.000000,0.000000,0.012987
4,1,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.013889,0.000000,0.000000,0.000000,0.013889,0.000000,0.000000


In [48]:
grouped.head()

,Cluster Labels,Borough,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,1,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.009009,0.000000,0.000000,0.009009,0.000000,0.000000,0.000000,0.000000,0.009009
1,0,Downtown Toronto,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.001553,0.001553,...,0.002329,0.013199,0.002329,0.000776,0.003882,0.000000,0.006211,0.000776,0.000776,0.002329
2,0,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024590
3,1,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000,0.000000,0.000000,0.012987
4,1,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.013889,0.000000,0.000000,0.000000,0.013889,0.000000,0.000000


In [58]:
data_merge = df_merge
data_merge = data_merge.join(grouped.set_index('Borough'), on='Borough')
data_merge.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A,North York,Parkwoods,43.7533,-79.3297,1,0.003984,0.000000,0.000000,0.003984,...,0.000000,0.000000,0.003984,0.003984,0.007968,0.0,0.000000,0.003984,0.011952,0.000000
1,M4A,North York,Victoria Village,43.7259,-79.3156,1,0.003984,0.000000,0.000000,0.003984,...,0.000000,0.000000,0.003984,0.003984,0.007968,0.0,0.000000,0.003984,0.011952,0.000000
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6543,-79.3606,0,0.000776,0.000776,0.000776,0.000776,...,0.002329,0.013199,0.002329,0.000776,0.003882,0.0,0.006211,0.000776,0.000776,0.002329
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7185,-79.4648,1,0.003984,0.000000,0.000000,0.003984,...,0.000000,0.000000,0.003984,0.003984,0.007968,0.0,0.000000,0.003984,0.011952,0.000000
4,M7A,Queen's Park,Queen's Park,43.6623,-79.3895,4,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.025641,0.000000,0.000000,0.000000,0.0,0.000000,0.025641,0.000000,0.025641


In [61]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
m = folium.Map(location=[latitude, longitude],zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(data_merge['Latitude'], data_merge['Longitude'], data_merge['Borough'], data_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(m)
       
m